In [ ]:
conda install -c conda-forge nilearn

In [1]:
import pandas as pd
import os
import urllib.request
import pandas as pd

In [9]:
import numpy as np

# Read the CSV file
data = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv')

# Define the output directory for saving the NIfTI files
output_dir = 'nifti_files'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Initialize the numpy array
num_samples = len(data)
array = np.zeros((num_samples, 2), dtype=np.int32)
matrices_par = []


In [16]:
# x = 0
# # Download and process each file
# for i, row in data.iterrows():
#     if x > 64 and x < 500:
#         file_id = row['FILE_ID']
#         dx_group = row['DX_GROUP']

#         if file_id != 'no_filename':
#             # Download the file
#             url = f'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/func_minimal/{file_id}_func_minimal.nii.gz'
#             file_path = os.path.join(output_dir, f'{file_id}.nii.gz')
#             urllib.request.urlretrieve(url, file_path)
#         print(file_id, " added")
#     x+=1
# # Print the numpy array


In [26]:
x = 0
# Download and process each file
for i, row in data.iterrows():
    if x > 64 and x < 500:
        file_id = row['FILE_ID']
        dx_group = row['DX_GROUP']

        if file_id != 'no_filename':
            # Download the file
            url = f'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/func_minimal/{file_id}_func_minimal.nii.gz'
#             file_path = os.path.join(output_dir, f'{file_id}.nii.gz')
            urllib.request.urlretrieve(url, file_path)
            image = nib.load(file_path)
            partial_correlation_matrix = partial_corr(image)
            matrices_par.append(partial_correlation_matrix)  
        print(file_id, " added")
        if os.path.exists(file_path):
            os.remove(file_path)
            print(file_id, " deleted")
    x+=1
# Print the numpy array


C:\Users\User\AppData\Roaming\Python\Python37\site-packages\joblib\memory.py:349: FutureWarning: The default strategy for standardize is currently 'zscore' which incorrectly uses population std to calculate sample zscores. The new strategy 'zscore_sample' corrects this behavior by using the sample std. In release 0.13, the default strategy will be replaced by the new strategy and the 'zscore' option will be removed. Please use 'zscore_sample' instead.
  return self.func(*args, **kwargs)


Olin_0050110  added
Olin_0050110  deleted


KeyboardInterrupt: 

In [24]:
x = 0
# Download and process each file
for i, row in data.iterrows():
    if x > 64 and x < 500:
        file_id = row['FILE_ID']
        dx_group = row['DX_GROUP']

        if file_id != 'no_filename':
            # Download the file
            url = f'https://s3.amazonaws.com/fcp-indi/data/Projects/ABIDE_Initiative/Outputs/cpac/func_minimal/{file_id}_func_minimal.nii.gz'
            file_path = os.path.join(output_dir, f'{file_id}.nii.gz')
            urllib.request.urlretrieve(url, file_path)
            print(file_id, " added")
            image = nib.load(file_path)
            partial_correlation_matrix = partial_corr(image)
            matrices_par.append(partial_correlation_matrix)  
            # Delete the downloaded file
#             if os.path.exists(file_path):
#                 os.remove(file_path)
#                 print(file_id, " deleted")
        x += 1


In [25]:
import glob
import nibabel as nib
from nilearn import image, input_data
from scipy import stats

def partial_corr(image_file):
    # Load the atlas image
    atlas_file = 'BN_Atlas_246_1mm.nii.gz'  # Replace with your atlas file
    atlas_data = nib.load(atlas_file).get_fdata()

    # Create a masker to extract ROI signals
    masker = input_data.NiftiLabelsMasker(labels_img=atlas_file, standardize=True)

    # Extract ROI signals from the image data
    roi_signals = masker.fit_transform(image_file)
    # Compute the partial correlation matrix
    partial_corr_matrix = np.zeros((len(roi_signals.T), len(roi_signals.T)))
    for i in range(len(roi_signals.T)):
        for j in range(i+1, len(roi_signals.T)):
            # Calculate the partial correlation coefficient
            r_partial, _ = stats.pearsonr(roi_signals.T[i], roi_signals.T[j])

            # Perform Fisher transformation to convert to a z-score
            z_partial = 0.5 * np.log((1 + r_partial) / (1 - r_partial))

            # Calculate the p-value
            p_value = 2 * stats.norm.cdf(-np.abs(z_partial))

            # Store the partial correlation coefficient in the matrix
            partial_corr_matrix[i, j] = r_partial
            partial_corr_matrix[j, i] = r_partial
    return partial_corr_matrix

# The partial correlation matrix will have shape (n_ROIs, n_ROIs)


In [13]:
matrices_par

[]